In [ ]:
%pip install pymysql

In [ ]:
import pymysql

- `.env` 파일을 읽어 envs라는 딕셔너리에 DB_HOST, DB_USER 등 환경 변수 저장

In [ ]:
# 1. '.env'파일을 읽어와서, 키=값으로 이뤄진 딕셔너리 생성
envs = {}

with open('.env', 'r') as f:
    for l in f.readlines():
        k, v = l.rstrip().split('=')
        envs[k] = v
        
envs    

- `envs`를 이용해 pymysql로 DB 연결 (conn 객체 생성)

In [ ]:
# 2. 접속정보 envs 활용해서 넣기 (port는 정수형으로)
conn = pymysql.connect(
    host=envs['DB_HOST'], # 19.222.124.233
    port=int(envs['DB_PORT']),
    user=envs['DB_USER'], 
    password=envs['DB_PASSWORD'], 
    database=envs['DB_DATABASE'], 
    charset='utf8mb4'
)

- `envs`에서 DB_로 시작하는 값만 추출해 db_config 딕셔너리로 정리 후 DB 재연결

In [ ]:
# 3. envs를 활용하여 db 정보만 담고있는 db_config를 만든 뒤, 다음의 코드가 실행되도록 하기
db_config = {} 
# db_config = {
#     'host': envs['DB_HOST'],
#     'port': int(envs['DB_PORT']),
#     'user': envs['DB_USER'],
#     'password': envs['DB_PASSWORD'],
#     'database': envs['DB_DATABASE']
# }

for k, v in envs.items():
    print(k,v)

    if 'DB' not in k: # 키에 db가 없으면 넘어가고
        continue
    # _로 분리하여 connect에 필요한 인자이름으로 바꿔줌 (host, port, user, password, database)
    k = k.split('_')[1].lower()
    print(k,v)
    # port 인 경우, v에 대해 int 처리
    if k == 'port':
        v = int(v)
        
    db_config[k] = v

conn = pymysql.connect(**db_config)
db_config

In [ ]:
envs

- `INSERT` 문을 실행하여 학생 1명 추가

In [ ]:
with conn.cursor() as cur:
    result = cur.execute('INSERT INTO Students (Name, Age, Address) VALUES (%s, %s, %s);', # SQL
                         ('david', 34, '경기') #Value
                         )
conn.commit()

- `executemany()`로 여러 학생을 한 번에 추가

In [ ]:
# 한 SQL문을 여러번 실행하고 싶을 때 executemany
with conn.cursor() as cur:
    result = cur.executemany('INSERT INTO Students (Name, Age, Address) VALUES (%s, %s, %s);', # SQL
                         [('david', 34, '경기'),('Joe', 22, '서울'),('zero', 17, '서울')] #Value
                         )
conn.commit()

- `SELECT * FROM Students`로 전체 데이터 조회 (DictCursor로 딕셔너리 형태 반환)

In [ ]:
import pymysql.cursors

with conn.cursor(cursor=pymysql.cursors.DictCursor) as cur:
    result = cur.execute('SELECT * FROM Students')
    datas = cur.fetchall() # fetchmany(size), fetchone()
result

In [ ]:
datas

- `insert_student()` 함수를 통해 재사용 가능한 삽입 기능 정의

In [ ]:
def insert_student(name, age, address):
    with conn.cursor() as cur:
        result = cur.execute(
            'INSERT INTO Students (Name, Age, Address) VALUES (%s, %s, %s);', # SQL
            (name, age, address))
    conn.commit()
insert_student('이름', 22, '')

- `conn.close()`로 연결 종료

In [ ]:
conn.close()